In [1]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" pip install llama-cpp-python --no-index --find-links=file:///kaggle/input/llama-cpp-wheels/llamacpp

Looking in links: file:///kaggle/input/llama-cpp-wheels/llamacpp
Processing /kaggle/input/llama-cpp-wheels/llamacpp/llama_cpp_python-0.2.25.tar.gz
  Installing build dependencies ... - \ | / done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... - \ done
  Preparing metadata (pyproject.toml) ... - done
Processing /kaggle/input/llama-cpp-wheels/llamacpp/diskcache-5.6.3-py3-none-any.whl (from llama-cpp-python)
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.25-cp310-cp310-manylinux_2_35_x86_64.whl size=8210094 sha256=c2e2acba53fa9b453169ee47e727bfb4a2056eb9f1874cdcef767f9f9b51cc46
  Stored in directory: /root/.cache/pip/wheels/45/6d/fa/bb7e80f98dd8f8d64c7ac7e906bbf6bf21ea20de0dfca6039c
Successfully built llama-cpp-python


In [2]:
import os
import polars as pl
from tqdm import tqdm
from llama_cpp import Llama

In [3]:
from huggingface_hub import snapshot_download

model_name = "mradermacher/YandexGPT-5-Lite-8B-instruct-GGUF"  # Пример модели
model_path = snapshot_download(
    repo_id=model_name,
    allow_patterns=["*Q8_0.gguf"],  # Скачивать только нужные файлы
    local_dir="./model",
    local_dir_use_symlinks=False,
    revision="main"  # Ветка репозитория
)
print(f"Модель сохранена в: {model_path}")

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

YandexGPT-5-Lite-8B-instruct.Q8_0.gguf:   0%|          | 0.00/8.54G [00:00<?, ?B/s]

Модель сохранена в: /kaggle/working/model


In [4]:
prompt = '''Тебе на вход будет дан комментарий из соц. сети для инвесторов "Пульс". Твой задача указать с помощью индексов, относится ли данный комментарий к ключевой ставке ЦБ РФ. В комментарии не обязательно упоминается ЦБ РФ, но под ставкой может все равно подразумеваться ключевая ставка.
Также могут быть комментарии, где упоминаются другие ставки, например, ставка оплаты, налоговая ставка, ставки иностранных компаний (в таких постах тогда часто встречается тикер иностранной компании), ставка как в казино. Такие ставки мне неинтересны, надо выделять комментарии именно, где подразумевается ставка ЦБ РФ.
Помимо того, что тебе необходимо выделить комментарии про ставку, тебе также необходимо сказать, в какую сторону изменится ставка ЦБ РФ, если об этом сказано в комментарии. Ставка может вырасти, не измениться или снизиться. Однако непосредственно данных слов может не быть в комментарии, возможны варианты.
Если по тексту не ясно, в каком направлении изменится ставка, так и напиши.

Какие индексы возваращать (то, что должно быть в ответе):
Если текст не относится к ставке ЦБ РФ верни число 0. И больше ничего не надо. То есть никакого дополнительного текста.
Если текст содержит информацию про ставку ЦБ РФ, в таком случае тебе еще необходимо опеределить направление изменения ставки. Тогда итоговый ответ необходимо представить в формате двух чисел через запятую между ними:
1) Первое число — верни 1, так как данный текст про ставку ЦБ РФ.
2) Второе число — если говорится, что ставку снизят, то -1, если не изменят, то 0, если повысят, то 1. Если же по тексту нельзя сказать, как изменится ставка, то верни 2.
То есть данный комментарий про ставку ЦБ РФ и по нему можно сказать, что автор ожидает повышение ставки ЦБ. Для этого случая от тебя требуется два числа через запятую и больше ничего.
Важно: Больше ничего, кроме одного или двух чисел через запятую не возвращай. Не надо пояснять свой ответ, не надо давать дополнительные комментарии, не надо дублировать слова из исходного комментария, нужны просто число или числа и больше ничего.

Комментарий:

'''

In [5]:
dfs = {}
files = [f.replace('.parquet', '') for f in os.listdir('/kaggle/input/dengi-i-credit') if 'retry' in f]
for file in files:
    dfs[file] = pl.read_parquet(f'/kaggle/input/dengi-i-credit/{file}.parquet')

In [6]:
# import asyncio
# from llama_cpp import Llama
# import torch
# from tqdm.asyncio import tqdm_asyncio

# class GPUTaskWorker:
#     def __init__(self, model_path, gpu_id, n_gpu_layers=-1):
#         self.gpu_id = gpu_id
#         self.model = Llama(
#             model_path=model_path,
#         	# repo_id="yandex/YandexGPT-5-Lite-8B-instruct-GGUF",
#         	# filename="YandexGPT-5-Lite-8B-instruct-Q4_K_M.gguf",
#             n_gpu_layers=n_gpu_layers,
#             n_threads=4,
#             n_ctx=1024,
#             main_gpu=gpu_id,
#         )
#         # print(f"Модель на GPU {gpu_id} использует {self.model._model.n_gpu_layers()} слоёв на GPU")
#         self.queue = asyncio.Queue()
#         self.is_working = False

#     async def process_queue(self):
#         while True:
#             prompt, future = await self.queue.get()
#             self.is_working = True
#             try:
#                 result = await asyncio.get_event_loop().run_in_executor(
#                     None,
#                     lambda: self.model(prompt, max_tokens=16)
#                 )
#                 future.set_result((self.gpu_id, result))
#             except Exception as e:
#                 future.set_exception(e)
#             finally:
#                 self.is_working = False
#                 self.queue.task_done()

# class MultiGPULlamaDispatcher:
#     def __init__(self, model_path, n_gpus=-1, n_gpu_layers=-1):
#         self.model_path = model_path
#         self.n_gpus = n_gpus if n_gpus > 0 else torch.cuda.device_count()
#         self.n_gpu_layers = n_gpu_layers
#         self.workers = []
#         self.loop = asyncio.get_event_loop()
        
#     async def initialize(self):
#         # Инициализация воркеров для каждого GPU
#         for gpu_id in range(self.n_gpus):
#             worker = GPUTaskWorker(self.model_path, gpu_id, self.n_gpu_layers)
#             self.workers.append(worker)
#             # Запускаем обработчик очереди для каждого воркера
#             asyncio.create_task(worker.process_queue())
    
#     async def dispatch_prompt(self, prompt):
#         # Находим наименее загруженный GPU
#         available_workers = [w for w in self.workers if not w.is_working]
#         if not available_workers:
#             # Если все заняты, выбираем тот, у которого самая короткая очередь
#             worker = min(self.workers, key=lambda w: w.queue.qsize())
#         else:
#             worker = available_workers[0]
        
#         future = self.loop.create_future()
#         await worker.queue.put((prompt, future))
#         return future
    
#     async def dispatch_prompts(self, prompts):
#         tasks = [self.dispatch_prompt(prompt) for prompt in prompts]
#         return await asyncio.gather(*tasks)

In [7]:
# model_path = '/kaggle/working/model'
# model_path = model_path + '/YandexGPT-5-Lite-8B-instruct-Q4_K_M.gguf'
# dispatcher = MultiGPULlamaDispatcher(model_path, n_gpus=2)
# await dispatcher.initialize()

# all_results = []

# for key in dfs:
#     responses = []
#     reasons = []
#     prompts = [prompt + message for message in dfs[key][key].to_list()[: 4]]
    
#     print(f"Начали обработку {len(prompts)} промптов на {dispatcher.n_gpus} GPU...")
    
#     results = await dispatcher.dispatch_prompts(prompts)
    
#     for result in results:
#         responses.append(result.result()[1]['choices'][0]['text'].strip())
#         reasons.append(result.result()[1]['choices'][0]['finish_reason'])
#     dfs[key] = dfs[key].wirh_columns(responses=responses, reasons=reasons)
#     dfs[key].write_parquet(f'{key}.parquet')

In [8]:
model_path = '/kaggle/working/model'
model_path = model_path + '/YandexGPT-5-Lite-8B-instruct.Q8_0.gguf'
llm = Llama(
    model_path=model_path,
    n_gpu_layers=-1,
    n_threads=4,
    n_ctx=4096,
)

ggml_init_cublas: GGML_CUDA_FORCE_MMQ:   no
ggml_init_cublas: CUDA_USE_TENSOR_CORES: yes
ggml_init_cublas: found 1 CUDA devices:
  Device 0: Tesla P100-PCIE-16GB, compute capability 6.0
llama_model_loader: loaded meta data with 44 key-value pairs and 291 tensors from /kaggle/working/model/YandexGPT-5-Lite-8B-instruct.Q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = YandexGPT 5 Lite 8B Instruct
llama_model_loader: - kv   3:                           general.finetune str              = instruct
llama_model_loader: - kv   4:                           general.basename str              = YandexGPT-5-Lite
llama_model_loader:

In [9]:
for key in dfs:
    responses = []
    reasons = []
    prompts = [prompt + message for message in dfs[key][key.replace('_retry', '')].to_list()]

    for prompt in tqdm(prompts):
        result = llm(prompt, max_tokens=128, echo=False)
        responses.append(result['choices'][0]['text'].strip())
        reasons.append(result['choices'][0]['finish_reason'])
    dfs[key] = dfs[key].with_columns(responses=pl.Series(responses), reasons=pl.Series(reasons))
    dfs[key].write_parquet(f'{key}.parquet')

  5%|▌         | 1/20 [00:11<03:46, 11.91s/it]
llama_print_timings:        load time =    5159.54 ms
llama_print_timings:      sample time =     120.54 ms /    60 runs   (    2.01 ms per token,   497.75 tokens per second)
llama_print_timings: prompt eval time =    5158.98 ms /   454 tokens (   11.36 ms per token,    88.00 tokens per second)
llama_print_timings:        eval time =    5953.64 ms /    59 runs   (  100.91 ms per token,     9.91 tokens per second)
llama_print_timings:       total time =   11904.47 ms
Llama.generate: prefix-match hit
 10%|█         | 2/20 [00:13<01:43,  5.75s/it]
llama_print_timings:        load time =    5159.54 ms
llama_print_timings:      sample time =      15.55 ms /     8 runs   (    1.94 ms per token,   514.54 tokens per second)
llama_print_timings: prompt eval time =     635.55 ms /    41 tokens (   15.50 ms per token,    64.51 tokens per second)
llama_print_timings:        eval time =     699.20 ms /     7 runs   (   99.89 ms per token,    10.01 toke